In [1]:
!conda env list

# conda environments:
#
base                     /mnt/data/hannan/miniconda3
contrastive              /mnt/data/hannan/miniconda3/envs/contrastive
deit                     /mnt/data/hannan/miniconda3/envs/deit
myenvi                *  /mnt/data/hannan/miniconda3/envs/myenvi
steganogan               /mnt/data/hannan/miniconda3/envs/steganogan



In [2]:
!pwd

/mnt/data/hannan/alaska/notebooks


In [3]:
import sys
sys.path.insert(0, "..")
import os
os.chdir("..")

In [4]:
import steganogan

In [5]:
from steganogan import SteganoGAN
steganogan = SteganoGAN.load(architecture='dense')

/mnt/data/hannan/miniconda3/envs/myenvi/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'steganogan.decoders.DenseDecoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/mnt/data/hannan/miniconda3/envs/myenvi/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'steganogan.critics.BasicCritic' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [7]:
steganogan.encode('alaska2-image-steganalysis/Cover/00005.jpg', 'alaska2-image-steganalysis/00005.png', 'This is a super secret message!')

In [8]:
try:
    secret = steganogan.decode('alaska2-image-steganalysis/SteganoGAN/00002.png')
except ValueError as ve:
    secret = ve
secret

'D4goZ3gLZLyHC6Gb5CieVT'

In [ ]:
try:
    secret = steganogan.decode('alaska2-image-steganalysis/SteganoGAN/00005.png')
except ValueError as ve:
    secret = str(ve)
secret

In [ ]:
!ls

In [ ]:
import random
import string

def get_random_alphanumeric_string(length):
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
#     print("Random alphanumeric String is:", result_str)
    return result_str

### Encode Secret

In [ ]:
import os
i = 0
for filename in sorted(os.listdir('alaska2-image-steganalysis/Cover/')):
    if i==200:
        break
    if i%100==0:
        print(filename)
    i = i + 1
    steganogan.encode(f'alaska2-image-steganalysis/Cover/{filename}',
                      f"alaska2-image-steganalysis/SteganoGAN/{filename.replace('jpg', 'png')}",
                      get_random_alphanumeric_string(random.randint(10, 50)))

### Decode secret

In [7]:
import glob
names = []
secrets = []
import os
i = 0
for filename in sorted(glob.glob('alaska2-image-steganalysis/SteganoGAN_Attk/*.png')):
    if i==10:
        break
    if i%10==0:
        print(filename)
    i = i + 1
    
    try:
        secret = steganogan.decode(filename)
    except ValueError as ve:
        secret = ve
    print(filename.split('/')[-1], secret)
    names.append(filename.split('/')[-1])
    secrets.append(secret)
names, secrets

alaska2-image-steganalysis/SteganoGAN_Attk/00742.png
00742.png ciZA9Lgt8odZwdPqXGoP2RsX84
00742_BIM.png Failed to find message.
00742_CW.png ciZA9Lgt8odZwdPqXGoP2RsX84
00742_FFGSM.png Failed to find message.
00742_FGSM.png Failed to find message.
00742_MIFGSM.png Failed to find message.
00742_PGD.png Failed to find message.
00742_RFGSM.png Failed to find message.
00742_TPGD.png Failed to find message.
01340.png CKWVUI7lmEaU2fTyQHypkcjYTkoK


(['00742.png',
  '00742_BIM.png',
  '00742_CW.png',
  '00742_FFGSM.png',
  '00742_FGSM.png',
  '00742_MIFGSM.png',
  '00742_PGD.png',
  '00742_RFGSM.png',
  '00742_TPGD.png',
  '01340.png'],
 ['ciZA9Lgt8odZwdPqXGoP2RsX84',
  ValueError('Failed to find message.'),
  'ciZA9Lgt8odZwdPqXGoP2RsX84',
  ValueError('Failed to find message.'),
  ValueError('Failed to find message.'),
  ValueError('Failed to find message.'),
  ValueError('Failed to find message.'),
  ValueError('Failed to find message.'),
  ValueError('Failed to find message.'),
  'CKWVUI7lmEaU2fTyQHypkcjYTkoK'])

In [21]:
img_names, img_attk = [], []
for name in names:
    split = name.split('.')[0].split('_')
    if len(split)==1:
        img_names.append(split[0])
        img_attk.append('None')
    else:
        img_names.append(name.split('.')[0].split('_')[0] )
        img_attk.append(name.split('.')[0].split('_')[1] )

In [23]:
import pandas as pd
data = {'names': img_names, 'attk':img_attk, 'secrets': secrets}
data = pd.DataFrame.from_dict(data)
data
data.to_csv('Evaluation/secrets.csv', index=False)  

### Investigate Decode secret 

In [37]:
from imageio import imread, imwrite
import torch
from collections import Counter
from steganogan.utils import bits_to_bytearray, bytearray_to_text, ssim, text_to_bits
image = 'alaska2-image-steganalysis/SteganoGAN/00742.png'

if not os.path.exists(image):
    raise ValueError('Unable to read %s.' % image)
    
image = imread(image, pilmode='RGB') / 255.0
image = torch.FloatTensor(image).permute(2, 1, 0).unsqueeze(0)
image = image.to(torch.device('cuda'))

image = steganogan.decoder(image).view(-1) > 0
candidates = Counter()
bits = image.data.cpu().numpy().tolist()

for candidate in bits_to_bytearray(bits).split(b'\x00\x00\x00\x00'):
    candidate = bytearray_to_text(bytearray(candidate))
    print(candidate)

False
False
False
False
ciZA9Lgt8odZwdPqXGoP2RsX84
ciZA9Lgt8odZwdPqXGoP2RsX84
ciZA9Lgt8odZwdPqXGoP2RsX84
ciZA9Lgt8odZwdPqXGoP2RsX84
ciZA9Lgt8odZwdPqXGoP2RsX84
ciZA9Lgt8odZwdPqXGoP2RsX84
ciZA9Lgt8odZwdPqXGoP2RsX84
ciZA9Lgt8odZwdPqXGoP2RsX84
ciZA9Lgt8odZwdPqXGoP2RsX84
ciZA9Lgt8odZwdPqXGoP2RsX84
ciZA9Lgt8odZwdPqXGoP2RsX84
False
False
False
False
False


In [38]:
from reedsolo import RSCodec
from imageio import imread, imwrite
import torch
from collections import Counter
from steganogan.utils import bits_to_bytearray, bytearray_to_text, ssim, text_to_bits

rs = RSCodec(250)
image = 'alaska2-image-steganalysis/SteganoGAN_Attk/00742_MIFGSM.png'

if not os.path.exists(image):
    raise ValueError('Unable to read %s.' % image)
    
image = imread(image, pilmode='RGB') / 255.0
image = torch.FloatTensor(image).permute(2, 1, 0).unsqueeze(0)
image = image.to(torch.device('cuda'))

image = steganogan.decoder(image).view(-1) > 0
candidates = Counter()
bits = image.data.cpu().numpy().tolist()
for candidate in bits_to_bytearray(bits).split(b'\x00\x00\x00\x00'):
#     candidate = bytearray_to_text(bytearray(candidate))
    text = rs.decode(bytearray(candidate))
    print(text)
    break

ReedSolomonError: Could not locate error